In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv')


In [ ]:
# preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [ ]:
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
from sklearn.preprocessing import OneHotEncoder 
onehotencoder_geo = OneHotEncoder()
geo_encoder = onehotencoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

In [ ]:
onehotencoder_geo.get_feature_names_out(['Geography'])
geo_encoded_df = pd.DataFrame(geo_encoder, columns =onehotencoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

In [ ]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

In [18]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehotencoder_geo,file)


In [19]:
#dependent and independent variables
X = data.drop(['Exited'], axis=1)
Y = data['Exited']  
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file) 

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


In [24]:
X_train.shape[1]

12

In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ##Hiden layer
    Dense(32, activation='relu'), ##Hiden layer2
    Dense(1,activation='sigmoid') ##Output layer
])
model.summary()

In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
import datetime
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = 'logs/fit/' +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
# set up early stopping 
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [39]:
# train the model
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=100,
    batch_size=32,  
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8697 - loss: 0.2991 - val_accuracy: 0.8640 - val_loss: 0.3427
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8673 - loss: 0.3131 - val_accuracy: 0.8620 - val_loss: 0.3396
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8675 - loss: 0.3180 - val_accuracy: 0.8660 - val_loss: 0.3387
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8707 - loss: 0.3069 - val_accuracy: 0.8560 - val_loss: 0.3446
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8677 - loss: 0.3074 - val_accuracy: 0.8590 - val_loss: 0.3425
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8712 - loss: 0.3073 - val_accuracy: 0.8570 - val_loss: 0.3445
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8719 - loss: 0.2993 - val_accuracy: 0.8570 - val_loss: 0.3443
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8706 - loss: 0.3093 - val_accu

In [40]:
model.save('model.h5')

In [43]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir logs/fit